# 这是一个汇总文档

## 文档的创建和实例

In [21]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField
from mongoengine import EmbeddedDocument, EmbeddedDocumentField, LazyReferenceField
from datetime import datetime

In [2]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### 查询要使用的全字段文档

In [13]:
class BookType(Document):
    type_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [4]:
BookType(type_name="经济类").save()
BookType(type_name="文学类").save()
BookType(type_name="计算机").save()

<BookType: BookType object>

In [19]:
class BookAuthor(Document):
    name = StringField()
    age = StringField()
    meta = {"db_alias": TEST_DB1}

In [20]:
class Seller(Document):
    name = StringField()
    phone = StringField(regex="\d")
    meta = {"db_alias": TEST_DB1}

In [23]:
class PublishingHouse(EmbeddedDocument):
    name =  StringField()
    phone =  StringField()

In [27]:
class BookAllField(Document):
    bookid = StringField(unique=True)
    name = StringField()
    ifnewbook = BooleanField()  # 是否为新上架图书
    shelves_date = DateTimeField()
    author = ReferenceField(BookAuthor)  # 使引用字段执行其他集合
    author_email = EmailField()
    price = FloatField()
    book_type = ReferenceField(BookType, dbref=True)  # 将引用ref文档
    publishing_house = EmbeddedDocumentField(PublishingHouse)
    seller = LazyReferenceField(Seller, dbref=True)
    tag = ListField()
    meta = {"db_alias": TEST_DB1}

In [28]:
BookAllField(bookid="wx0927615ssr", name = "大学", ifnewbook = False, 
        shelves_date = datetime.strptime("2020/12/20","%Y/%m/%d"),
        price = 10.8, tag = ["历史", "经典"], author_email = "zengzi@qq.com",
             author = BookAuthor(name="曾子", age="春秋").save(),
             seller = Seller(name="当当书店", phone="1288753").save(),
             book_type = BookType.objects(type_name="文学类").first(),
             publishing_house = PublishingHouse(name="人民教育出版社", phone="120934876")
    ).save()

<BookAllField: BookAllField object>

In [26]:
BookAllField(bookid="wx09s15ssr", name = "中庸", ifnewbook = False, 
        shelves_date = datetime.strptime("2020/11/20","%Y/%m/%d"),
        price = 10.8, tag = ["历史", "经典"], author_email = "zengzi@qq.com",
             author = BookAuthor(name="孔伋", age="春秋").save(),
             seller = Seller(name="当当书店", phone="128875ds3").save(),
             book_type = BookType.objects(type_name="文学类").first(),
             publishing_house = PublishingHouse(name="人民教育出版社", phone="120934876")
    ).save()

<BookAllField: BookAllField object>